# LangChain: Memory

## Outline
* ConversationBufferMemory
* ConversationBufferWindowMemory
* ConversationTokenBufferMemory
* ConversationSummaryMemory

## ConversationBufferMemory


In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [41]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [42]:
conversation.predict(input="Hi my name is Vahid.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi my name is Vahid.
AI:

> Finished chain.


"Hello Vahid! It's nice to meet you. How can I assist you today?"

In [43]:
conversation.predict(input="what is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi my name is Vahid.
AI: Hello Vahid! It's nice to meet you. How can I assist you today?
Human: what is 1+1?
AI:

> Finished chain.


'1+1 is equal to 2.'

In [44]:
conversation.predict(input="what is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi my name is Vahid.
AI: Hello Vahid! It's nice to meet you. How can I assist you today?
Human: what is 1+1?
AI: 1+1 is equal to 2.
Human: what is my name?
AI:

> Finished chain.


'Your name is Vahid.'

In [45]:
print(memory.buffer)

Human: Hi my name is Vahid.
AI: Hello Vahid! It's nice to meet you. How can I assist you today?
Human: what is 1+1?
AI: 1+1 is equal to 2.
Human: what is my name?
AI: Your name is Vahid.


In [46]:
memory.load_memory_variables({}) # Remember we should use {} when we want to call this variable

{'history': "Human: Hi my name is Vahid.\nAI: Hello Vahid! It's nice to meet you. How can I assist you today?\nHuman: what is 1+1?\nAI: 1+1 is equal to 2.\nHuman: what is my name?\nAI: Your name is Vahid."}

In [47]:
memory.save_context({"Human" : "Can you sing me a song?"}, {"AI": "Fuck you of course not"})

In [31]:
print(memory.buffer)

Human: Hi my name is Vahid.
AI: Hello Vahid! It's nice to meet you. How can I assist you today?
Human: what is 1+1?
AI: 1+1 is equal to 2.
Human: what is my name?
AI: Your name is Vahid.
Human: Can you sing me a song?
AI: Fuck you of course not


In [48]:
memory = ConversationBufferMemory()

In [49]:
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

In [51]:
print(memory.buffer)

Human: Hi
AI: What's up


In [52]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

In [53]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [54]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

## ConversationBufferWindowMemory

It allows the machine just remember the K number of previous conversation

In [55]:
from langchain.memory import ConversationBufferWindowMemory

In [56]:
memory = ConversationBufferWindowMemory(k=1)               

In [57]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

In [58]:
memory.load_memory_variables({})


{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [66]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

In [67]:
conversation.predict(input="Hi, my name is Andrew")

"Hello Andrew! It's nice to meet you. How can I assist you today?"

In [68]:
conversation.predict(input="What is 1+1?")

'1+1 is equal to 2.'

In [69]:
conversation.predict(input="What is my name?")

"I'm sorry, but I don't have access to personal information."

## ConversationTokenBufferMemory
We should specify the llm for the this tokenizer because different tokenisers have different ways of tokenising. 

In [81]:
#!pip install tiktoken
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0)

In [82]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=40)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [83]:
memory.load_memory_variables({})


{'history': 'Human: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

## ConversationSummaryMemory

In [85]:
from langchain.memory import ConversationSummaryBufferMemory

In [108]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})


In [113]:
memory.load_memory_variables({})

{'history': 'System: The human and AI exchange greetings. The human asks about the schedule for the day. The AI provides a detailed schedule, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of bringing a laptop to showcase the latest LLM demo during the lunch meeting.'}

In [114]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [115]:
conversation.predict(input="What would be a good demo to show?")




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI exchange greetings. The human asks about the schedule for the day. The AI provides a detailed schedule, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of bringing a laptop to showcase the latest LLM demo during the lunch meeting.
Human: What would be a good demo to show?
AI:

> Finished chain.


'A good demo to show during the lunch meeting with the customer interested in AI would be the latest LLM (Language Model) demo. The LLM is a cutting-edge AI model that can generate human-like text based on a given prompt. It has been trained on a vast amount of data and can generate coherent and contextually relevant responses. By showcasing the LLM demo, you can demonstrate the capabilities of our AI technology and how it can be applied to various industries and use cases.'

In [116]:
memory.load_memory_variables({})

{'history': 'System: The human and AI exchange greetings and discuss the schedule for the day. The AI provides a detailed schedule, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the importance of bringing a laptop to showcase the latest LLM demo during the lunch meeting. The human asks what would be a good demo to show, and the AI suggests showcasing the latest LLM (Language Model) demo. The LLM is a cutting-edge AI model that can generate human-like text based on a given prompt. It has been trained on a vast amount of data and can generate coherent and contextually relevant responses. By showcasing the LLM demo, the AI believes they can demonstrate the capabilities of their AI technology and how it can be applied to various industries and use cases.'}